In [1]:
from sqlalchemy import create_engine
import pandas as pd
from fuzzywuzzy import process, fuzz
import geopy
from geopy.geocoders import Nominatim
from os import environ
from googlemaps import Client
import math

The database URL contains credentials that should be protected, so we use an environment variable.

In [2]:
connection_url =  environ["ICEURL"]

In [3]:
dataframe = pd.read_sql_table("locations", connection_url, "public")

We scraped TRAC (http://trac.syr.edu) detention center location data in a different Jupyter notebook, and saved as a CSV file.

In [4]:
trac_locations = pd.read_csv('trac_locations_corrected.csv')
choices = trac_locations['Facility Name']

# We're going to use Nominatim (geolocator) and google maps (client)
geolocator = Nominatim(user_agent="iceicedata")
geopy.geocoders.options.default_timeout = 7

client = Client(environ["APIKEY"])


In [5]:
def enrich_row(old_row):
    row = old_row.copy()
    if math.isnan(row['lat']):
        state = ''
        city = ''
        lat = math.nan
        lng = math.nan
        if row['state'] is not None:
            state = row['state']
            if row['city'] is not None:
                city = row['city']
            else:
                city = ''
        else:
            name = row['name']
            match = process.extractOne(name, choices, scorer=fuzz.token_set_ratio)
            if match[1] > 92:
                city = trac_locations.at[match[2], 'City']
                state = trac_locations.at[match[2], 'State']
                location = geolocator.geocode(city + ', ' + state)
                if location is not None:
                    lng = location.longitude
                    lat = location.latitude
                else:
                    lng = math.nan
                    lat = math.nan

        if state == '':
            # Try google
            place = client.places(name)
            if place['status'] == 'OK':
                lat = place['results'][0]['geometry']['location']['lat']
                lng = place['results'][0]['geometry']['location']['lng']
                location = geolocator.reverse(str(lat) + ', ' + str(lng))
                if location is not None:
                    if 'city' in location.raw['address']:
                        city = location.raw['address']['city']
                    elif 'hamlet' in location.raw['address']:
                        city = location.raw['address']['hamlet']
                    elif 'town' in location.raw['address']:
                        city = location.raw['address']['town']
                    elif 'village' in location.raw['address']:
                        city = location.raw['address']['village']
                    elif 'locality' in location.raw['address']:
                        city = location.raw['address']['locality']
                    else:
                        city = ''
        if not math.isnan(lat):
            row['city'] = city
            row['state'] = state
            row['lon'] = lng
            row['lat'] = lat
    return row


In [6]:
dataframe.to_csv('ice_locations_original.csv', index=False)

In [7]:
# Add location information, if we can find it.

new_dataframe = dataframe.apply(enrich_row, axis=1)

In [8]:
new_dataframe.to_csv('ice_locations.csv', index=False)

In [59]:
# engine = create_engine(connection_url)
# engine.execute('DELETE from public.locations ')
# dataframe2 = new_dataframe.copy()
# for col in ['date_first_record', 'date_recent_record', 'created_at', 'updated_at' ]:
#     dataframe2[col] = (new_dataframe[col]).apply(lambda d: pd.to_datetime(str(d)))
# dataframe2.to_sql(name='locations', con=connection_url, if_exists='append', index=False)